In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score 
from sklearn.neural_network import MLPClassifier
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as functional

from torchmetrics import Precision, Recall

c:\Users\yaggod\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = fetch_openml('mnist_784', version=1)

X, y = dataset.data, dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [3]:
nn_model = MLPClassifier(hidden_layer_sizes=(100, 100), verbose=True, early_stopping=True)

nn_model.fit(X_train, y_train)

Iteration 1, loss = 2.64179595
Validation score: 0.896429
Iteration 2, loss = 0.70381040
Validation score: 0.917857
Iteration 3, loss = 0.41170614
Validation score: 0.930536
Iteration 4, loss = 0.29249145
Validation score: 0.936429
Iteration 5, loss = 0.21228374
Validation score: 0.938036
Iteration 6, loss = 0.16267215
Validation score: 0.945357
Iteration 7, loss = 0.14385067
Validation score: 0.945000
Iteration 8, loss = 0.11533649
Validation score: 0.948571
Iteration 9, loss = 0.11095733
Validation score: 0.951429
Iteration 10, loss = 0.10468257
Validation score: 0.956964
Iteration 11, loss = 0.09332705
Validation score: 0.949107
Iteration 12, loss = 0.09847620
Validation score: 0.951786
Iteration 13, loss = 0.08166395
Validation score: 0.954107
Iteration 14, loss = 0.08175055
Validation score: 0.950714
Iteration 15, loss = 0.08658054
Validation score: 0.954286
Iteration 16, loss = 0.09236546
Validation score: 0.956071
Iteration 17, loss = 0.07841191
Validation score: 0.954821
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 100), verbose=True)

In [4]:
predicted = nn_model.predict(X_test)


precision = precision_score(y_test, predicted, average = 'macro')
recall = recall_score(y_test, predicted, average = 'macro')
print("Precision:", precision)
print("Recall:", recall)

Precision: 0.970050446621182
Recall: 0.9698183428759455


In [5]:
torch_X_train = torch.from_numpy(X_train.values).type(torch.float).reshape(-1, 1, 28, 28).cuda()
torch_X_test = torch.from_numpy(X_test.values).type(torch.float).reshape(-1, 1, 28, 28).cuda()
torch_y_train = torch.from_numpy(y_train.to_numpy(dtype=int)).cuda()
torch_y_test = torch.from_numpy(y_test.to_numpy(dtype=int)).cuda()

train_tensor = torch.utils.data.TensorDataset(torch_X_train, torch_y_train)
test_tensor = torch.utils.data.TensorDataset(torch_X_test, torch_y_test)

In [6]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(train_tensor, batch_size = batch_size)
test_loader = torch.utils.data.DataLoader(test_tensor, batch_size = batch_size)

In [10]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.linear1 = nn.Linear(3*3*64, 100)
        self.output = nn.Linear(100, 10)
    
    def forward(self, X):
        X = functional.relu(self.conv1(X))
        X = functional.relu(functional.max_pool2d(self.conv2(X), 2))
        X = functional.relu(functional.max_pool2d(self.conv3(X), 2))
        X = X.view(-1,3*3*64 )
        X = functional.relu(self.linear1(X))
        X = self.output(X)
        
        return torch.log_softmax(X, dim=0)

    def fit(self, data, epochs = 20, learning_rate = 0.0001):
        optimizer = torch.optim.Adam(nn_model.parameters(), lr = learning_rate)
        loss_function = torch.nn.CrossEntropyLoss()
        self.train()
        for epoch in range(epochs):
            for x, y in data:
                predicted = self(x)
                optimizer.zero_grad()
                loss = loss_function(predicted, y)
                loss.backward()
                optimizer.step()
            print(epoch)
        nn_model.eval()

In [11]:
nn_model = ConvolutionalNeuralNetwork().cuda()
print(nn_model)

nn_model.fit(train_loader)

ConvolutionalNeuralNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=576, out_features=100, bias=True)
  (output): Linear(in_features=100, out_features=10, bias=True)
)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [12]:
predicted = nn_model(torch_X_test).argmax(dim=1)

precision_calculator = Precision(task="multiclass", num_classes=10, average='macro')
recall_calculator = Recall(task="multiclass", num_classes=10, average='macro')

precision = precision_calculator(predicted.cpu(), torch_y_test.cpu())
recall = recall_calculator(predicted.cpu(), torch_y_test.cpu())
print("Precision:", precision)
print("Recall:", recall)

Precision: tensor(0.9893)
Recall: tensor(0.9893)
